In [25]:
import ee
import datetime
import eemont

In [26]:
#ee.Authenticate()
ee.Initialize()

PARAMETERS

In [27]:
# Fields

altm_buf = ee.FeatureCollection('projects/ee-bmperezaraujo/assets/fields_altm_buffered')
lanmo_buf = ee.FeatureCollection('projects/ee-bmperezaraujo/assets/lanmo_buffered')
komo_buf = ee.FeatureCollection('projects/ee-bmperezaraujo/assets/KomoFields_new/old_komo_buf') #Komo for 2022 validation
dt_2023 = ee.FeatureCollection('projects/ee-bmperezaraujo/assets/DT_2023')
rt_2018 = ee.FeatureCollection('projects/ee-bmperezaraujo/assets/RT_2018')

In [28]:
# Areas of interest

rectangle_around_komo = [
    [12.24781567858654,48.63364077609223],
    [12.261896825004106,48.61616706415128],
    [12.284205163929942,48.60050609850866],
    [12.788165456118127,48.69214064873259],
    [12.808848918234043,48.72685336443358],
    [12.7883091278265,48.748023697709286],
    [12.739480336773559,48.7371718118958],
    [12.547310137926523,48.698203080639445],
    [12.24781567858654,48.63364077609223]
    ]

rectangle_around_altm = [
    [10.601607342048558,49.1400903899757],
    [10.724688549323949,49.1400903899757],
    [10.724688549323949,49.19979952240029],
    [10.601607342048558,49.19979952240029],
    [10.601607342048558,49.1400903899757]
    ]

rectangle_around_lanmo = [
    [11.182180706119182,48.617264539449096],
    [11.225267711734416,48.617264539449096],
    [11.225267711734416,48.64471993161285],
    [11.182180706119182,48.64471993161285],
    [11.182180706119182,48.617264539449096]
    ]

rectangle_around_altm_alesheim = [ # For AL_2023
    [10.612556349729182,49.1788311237893],
    [10.64156712243426,49.149534320681724],
    [10.67177952477801,49.14055098693633],
    [10.80361546227801,49.057827892005704],
    [10.835201155637385,49.02632094126456],
    [10.85580052087176,49.020917745442524],
    [10.87777317712176,49.03352428981247],
    [10.86129368493426,49.05692797040905],
    [10.711604964231135,49.18545137092539],
    [10.619251143430354,49.18455376219791],
    [10.61135472009051,49.18163642138161],
    [10.612556349729182,49.1788311237893]
]

rectangle_around_altm_2020_extra = [ # Extra AL 2020 birds
    [10.662692698733517,49.078907557316256],
    [10.812381419436642,49.078907557316256],
    [10.812381419436642,49.18269482730185],
    [10.662692698733517,49.18269482730185],
    [10.662692698733517,49.078907557316256]
]

rectangle_around_donaumoos = [ # Extra DM birds
    [11.163654736941853,48.60096794569677],
    [11.371021680301228,48.60096794569677],
    [11.371021680301228,48.70331543696478],
    [11.163654736941853,48.70331543696478],
    [11.163654736941853,48.60096794569677]
]

rectangle_around_rta = [ # Extra RTA bird
    [12.403543368324614,48.96554045023258],
    [12.45710171793399,48.96554045023258],
    [12.45710171793399,48.99686062344109],
    [12.403543368324614,48.99686062344109],
    [12.403543368324614,48.96554045023258]
]

rectangle_around_dota = [
    [12.655737662070639,48.74098225628875],
    [12.909453177207357,48.74098225628875],
    [12.909453177207357,48.91727115497619],
    [12.655737662070639,48.91727115497619],
    [12.655737662070639,48.74098225628875]
]

In [29]:
AOI = ee.Geometry.Polygon(rectangle_around_rta)
START_DATE = '2018-03-01'
END_DATE = '2018-07-31' #datetime.datetime.now().strftime('%Y-%m-%d') # current date
CLOUD_FILTER = 60 # Not in use
CLD_PRB_THRESH = 65
NIR_DRK_THRESH = 0.15
CLD_PRJ_DIST = 1
BUFFER = 7 # Mask buffer
MONTH_FILTER = ee.Filter.calendarRange(3, 7, 'month')

In [30]:
# Export parameters

fields = rt_2018 # Fields for median calculation and export
description = 'S2_RT_2018' # Name of file in Drive
fieldIdColumn = 'new_id' # Lanmo + Komo for validation: fid // Altm: field_id

FILTER ORIGINAL IC

In [31]:
def get_s2_sr_cld_col(aoi, start_date, end_date):
    # Import and filter S2 SR.
    s2_sr_col = (ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')
        .filterBounds(aoi)
        .filterDate(start_date, end_date)
        .filter(MONTH_FILTER)
#        .filter(ee.Filter.lte('CLOUDY_PIXEL_PERCENTAGE', CLOUD_FILTER))
        )

    # Import and filter s2cloudless.
    s2_cloudless_col = (ee.ImageCollection('COPERNICUS/S2_CLOUD_PROBABILITY')
        .filterBounds(aoi)
        .filterDate(start_date, end_date)
        .filter(MONTH_FILTER))

    # Join the filtered s2cloudless collection to the SR collection by the 'system:index' property.
    return ee.ImageCollection(ee.Join.saveFirst('s2cloudless').apply(**{
        'primary': s2_sr_col,
        'secondary': s2_cloudless_col,
        'condition': ee.Filter.equals(**{
            'leftField': 'system:index',
            'rightField': 'system:index'
        })
    }))

In [32]:
filtered_IC = get_s2_sr_cld_col(AOI, START_DATE, END_DATE)

CLOUD AND SHADOW MASKING

In [33]:
# Support function for cloud masking

def add_cloud_bands(img):
    # Get s2cloudless image, subset the probability band.
    cld_prb = ee.Image(img.get('s2cloudless')).select('probability')

    # Condition s2cloudless by the probability threshold value.
    is_cloud = cld_prb.gt(CLD_PRB_THRESH).rename('clouds')

    # Add the cloud probability layer and cloud mask as image bands.
    return img.addBands(ee.Image([cld_prb, is_cloud]))

In [34]:
# Support function for shadow masking

def add_shadow_bands(img):
    # Identify water pixels from the SCL band.
    not_water = img.select('SCL').neq(6)

    # Identify dark NIR pixels that are not water (potential cloud shadow pixels).
    SR_BAND_SCALE = 1e4
    dark_pixels = img.select('B8').lt(NIR_DRK_THRESH*SR_BAND_SCALE).multiply(not_water).rename('dark_pixels')

    # Determine the direction to project cloud shadow from clouds (assumes UTM projection).
    shadow_azimuth = ee.Number(90).subtract(ee.Number(img.get('MEAN_SOLAR_AZIMUTH_ANGLE')));

    # Project shadows from clouds for the distance specified by the CLD_PRJ_DIST input.
    cld_proj = (img.select('clouds').directionalDistanceTransform(shadow_azimuth, CLD_PRJ_DIST*10)
        .reproject(**{'crs': img.select(0).projection(), 'scale': 100})
        .select('distance')
        .mask()
        .rename('cloud_transform'))

    # Identify the intersection of dark pixels with cloud shadow projection.
    shadows = cld_proj.multiply(dark_pixels).rename('shadows')

    # Add dark pixels, cloud projection, and identified shadows as image bands.
    return img.addBands(ee.Image([dark_pixels, cld_proj, shadows]))

In [35]:
# Aggregate cloud and shadow mask into final function

def add_cld_shdw_mask(img):
    # Add cloud component bands.
    img_cloud = add_cloud_bands(img)

    # Add cloud shadow component bands.
    img_cloud_shadow = add_shadow_bands(img_cloud)

    # Combine cloud and shadow mask, set cloud and shadow as value 1, else 0.
    is_cld_shdw = img_cloud_shadow.select('clouds').add(img_cloud_shadow.select('shadows')).gt(0)

    # Remove small cloud-shadow patches and dilate remaining pixels by BUFFER input.
    # 20 m scale is for speed, and assumes clouds don't require 10 m precision.
    is_cld_shdw = (is_cld_shdw.focalMin(2).focalMax(BUFFER)
        .reproject(**{'crs': img.select([0]).projection(), 'scale': 10})
        .rename('cloudmask'))

    # Add the final cloud-shadow mask to the image.
    return img_cloud_shadow.addBands(is_cld_shdw)

In [36]:
# Apply cloud and shadow bands, clip to AOI

IC_withMaskBands = filtered_IC.map(add_cld_shdw_mask).map(lambda image: image.clip(AOI))

In [37]:
# Apply cloud and shadow mask based on 'cloudmask' band

def maskCloudsAndShadows(image):
    cloudmask = image.select('cloudmask')
    maskedImage = image.updateMask(cloudmask.eq(0))
    return maskedImage

In [38]:
# Mask out clouds and shadows

masked_IC = IC_withMaskBands.map(maskCloudsAndShadows)

In [39]:
# Calculate indices
# Will remove bands from mask calculation
# But doing it after ensures mask output is not changed

IC_withIndices = masked_IC.scaleAndOffset().spectralIndices(["NDVI", "SAVI", "EVI", "MSI", "NMDI", "MNDWI", "NDWI", "GNDVI"])

In [40]:
Final_IC = IC_withIndices

count = Final_IC.size()
print('Number of images in the collection:', count.getInfo())

Number of images in the collection: 116


EXPORT AS GEE ASSET

In [41]:
%% skip

# Export IC as asset

image_list = masked_IC.toList(masked_IC.size())

for i in range(image_list.length().getInfo()):
    image = ee.Image(image_list.get(i))
    
    export_params = {
        'image': image,
        'description': 'masked_image_export',
        'assetId': 'projects/ee-bmperezaraujo/assets/s2_cloud_shadow_masked/' + image.get('system:index').getInfo(),
        'scale': 10
    }
    
    task = ee.batch.Export.image.toAsset(**export_params)
    task.start()

UsageError: Cell magic `%%` not found.


In [42]:
%% skip

# Copy images to appropriate IC folder

src_folder = "projects/ee-bmperezaraujo/assets/"
dest_folder = "projects/ee-bmperezaraujo/assets/IC_komo_shadowmasked"
assets = ee.data.listAssets({'parent': src_folder})

for asset in assets['assets']:
    if 's2_cloud_shadow_masked' in asset['id']:
        new_asset = dest_folder + '/' + asset['id'].split('/')[-1]
        ee.data.copyAsset(asset['id'], new_asset, True)
        ee.data.deleteAsset(asset['id'])

UsageError: Cell magic `%%` not found.


EXPORT AS CSV

In [43]:
#%% skip

def exportTimeSeriesToCSV(description, s2CloudMasked, fieldIdColumn):
    region_list = fields.toList(fields.size())
  
    def exportFunction(ele):
        geometry = ee.Feature(ele).geometry()
        sen2_masked_export = s2CloudMasked.filterBounds(geometry)
    
        def getData(e):
            image = ee.Image(e)
            date = ee.Date(image.get("system:time_start")).format().slice(0, 10)
      
            region = image.reduceRegion(ee.Reducer.median(), geometry)
            
            ft = ee.Feature(None, {
                'date': date,
                'aerosol': region.get('B1'),
                'blue': region.get('B2'),
                'green': region.get('B3'),
                'red': region.get('B4'),
                're1': region.get('B5'),
                're2': region.get('B6'),
                're3': region.get('B7'),
                'nir': region.get('B8'),
                're4': region.get('B8A'),
                'water_vapour': region.get('B9'),
                'swir1': region.get('B11'),
                'swir2': region.get('B12'),
                'NDVI': region.get('NDVI'),
                'SAVI': region.get('SAVI'),
                'EVI': region.get('EVI'),
                'MSI': region.get('MSI'),
                'NMDI': region.get('NMDI'),
                'MNDWI': region.get('MNDWI'),
                'NDWI': region.get('NDWI'),
                'GNDVI': region.get('GNDVI'),
                'cloud_cover': image.get("CLOUDY_PIXEL_PERCENTAGE"),
                'field_id': ee.Feature(ele).get(fieldIdColumn)
            })
      
            return ft
    
        return sen2_masked_export.toList(sen2_masked_export.size()).map(getData).flatten()
  
    forExport = region_list.map(exportFunction)
  
    # Export the time-series as a CSV.
    task = ee.batch.Export.table.toDrive(
        collection=ee.FeatureCollection(forExport.flatten()),
        description=description,
        selectors=['date, aerosol, blue, green, red, re1, re2, re3, nir, re4, water_vapour, swir1, swir2, NDVI, SAVI, EVI, MSI, NMDI, MNDWI, NDWI, GNDVI, cloud_cover, field_id'],
        folder='extra_birds'
    )
    task.start()

In [44]:
#%% skip

exportTimeSeriesToCSV(description, Final_IC, fieldIdColumn)

VISUALISATION

In [379]:
%% skip

import folium

# Define a method for displaying Earth Engine image tiles to a folium map.
def add_ee_layer(self, ee_image_object, vis_params, name, show=True, opacity=1, min_zoom=0):
    map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
    folium.raster_layers.TileLayer(
        tiles=map_id_dict['tile_fetcher'].url_format,
        attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
        name=name,
        show=show,
        opacity=opacity,
        min_zoom=min_zoom,
        overlay=True,
        control=True
        ).add_to(self)

# Add the Earth Engine layer method to folium.
folium.Map.add_ee_layer = add_ee_layer

In [380]:
%% skip

# Simple visualisation

image = Final_IC.sort('CLOUDY_PIXEL_PERCENTAGE').first()
#image = Final_IC.filter(ee.Filter.date('2022-05-20', '2022-06-30')).first() # Low cloud density
#image = Final_IC.filter(ee.Filter.date('2022-05-25', '2022-05-27')).first() # Middle cloud density
#image = Final_IC.filter(ee.Filter.date('2022-06-25', '2022-06-30')).first() # High cloud density

# Get the image visualization parameters
vis_params = {
    'bands': ['B4', 'B3', 'B2'],
    'min': 0,
    'max': 0.3
}

#map = folium.Map(location=[48.66181704242688, 12.49038135956078], zoom_start=15) # Komo
map = folium.Map(location=[49.16304293215874, 10.691069976654566], zoom_start=15) # Altm
#map = folium.Map(location=[48.63508880956953, 11.218028674587813], zoom_start=15) # Lanmo

map.add_ee_layer(image, vis_params, 'Image')

display(map)

In [381]:
%% skip

def display_cloud_layers(col):
    # Mosaic the image collection.
    img = col.mosaic()

    # Subset layers and prepare them for display.
    clouds = img.select('clouds').selfMask()
    shadows = img.select('shadows').selfMask()
    dark_pixels = img.select('dark_pixels').selfMask()
    probability = img.select('probability')
    cloudmask = img.select('cloudmask').selfMask()
    cloud_transform = img.select('cloud_transform')

    # Create a folium map object.
    center = AOI.centroid(10).coordinates().reverse().getInfo()
    m = folium.Map(location=center, zoom_start=12)

    # Add layers to the folium map.
    m.add_ee_layer(img,
                   {'bands': ['B4', 'B3', 'B2'], 'min': 0, 'max': 2500, 'gamma': 1.1},
                   'S2 image', True, 1, 9)
    m.add_ee_layer(probability,
                   {'min': 0, 'max': 100},
                   'probability (cloud)', False, 1, 9)
    m.add_ee_layer(clouds,
                   {'palette': 'e056fd'},
                   'clouds', False, 1, 9)
    m.add_ee_layer(cloud_transform,
                   {'min': 0, 'max': 1, 'palette': ['white', 'black']},
                   'cloud_transform', False, 1, 9)
    m.add_ee_layer(dark_pixels,
                   {'palette': 'orange'},
                   'dark_pixels', False, 1, 9)
    m.add_ee_layer(shadows, {'palette': 'yellow'},
                   'shadows', False, 1, 9)
    m.add_ee_layer(cloudmask, {'palette': 'orange'},
                   'cloudmask', True, 0.5, 9)

    # Add a layer control panel to the map.
    m.add_child(folium.LayerControl())

    # Display the map.
    display(m)

UsageError: Cell magic `%%` not found.


In [382]:
%% skip

display_cloud_layers(masked_IC)

UsageError: Cell magic `%%` not found.
